<a href="https://colab.research.google.com/github/GabrielJobert/EU-27-trade-network-analysis/blob/main/M%C3%A9moire_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers datasets torchtext

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

class RetrieverSquadDataset(Dataset):
    def __init__(self, split='train', tokenizer_name='bert-base-uncased', max_length=384, include_negatives=False):
        # Load the SQuAD 2.0 dataset
        self.dataset = load_dataset('squad_v2', split=split)

        # Load the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

        # Set maximum length for tokenizer
        self.max_length = max_length

        # Include negatives if specified
        self.include_negatives = include_negatives

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Extract data at the specified index
        data = self.dataset[idx]

        # Tokenize the context (passage)
        passage_inputs = self.tokenizer(
            data['context'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the query (question)
        query_inputs = self.tokenizer(
            data['question'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Convert to single tensors
        passage_input_ids = passage_inputs['input_ids'].squeeze()
        passage_attention_mask = passage_inputs['attention_mask'].squeeze()

        query_input_ids = query_inputs['input_ids'].squeeze()
        query_attention_mask = query_inputs['attention_mask'].squeeze()

        # If including negatives, prepare a negative sample
        if self.include_negatives:
            # Find a random negative passage (simple implementation, could be more sophisticated)
            neg_idx = (idx + 1) % len(self.dataset)
            negative_data = self.dataset[neg_idx]
            negative_inputs = self.tokenizer(
                negative_data['context'],
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )
            negative_input_ids = negative_inputs['input_ids'].squeeze()
            negative_attention_mask = negative_inputs['attention_mask'].squeeze()

            return {
                'query_input_ids': query_input_ids,
                'query_attention_mask': query_attention_mask,
                'passage_input_ids': passage_input_ids,
                'passage_attention_mask': passage_attention_mask,
                'negative_input_ids': negative_input_ids,
                'negative_attention_mask': negative_attention_mask
            }

        # Return positive pairs
        return {
            'query_input_ids': query_input_ids,
            'query_attention_mask': query_attention_mask,
            'passage_input_ids': passage_input_ids,
            'passage_attention_mask': passage_attention_mask
        }

# Example usage
if __name__ == "__main__":
    retriever_dataset = RetrieverSquadDataset(split='train', include_negatives=True)
    dataloader = DataLoader(retriever_dataset, batch_size=8, shuffle=True)

    # Iterate over the DataLoader
    for batch in dataloader:
        print(batch)
        break

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'query_input_ids': tensor([[  101,  2129,  2116,  ...,     0,     0,     0],
        [  101,  2043,  2020,  ...,     0,     0,     0],
        [  101,  2029,  7185,  ...,     0,     0,     0],
        ...,
        [  101,  2054,  2106,  ...,     0,     0,     0],
        [  101,  2054, 11130,  ...,     0,     0,     0],
        [  101,  1996, 12723,  ...,     0,     0,     0]]), 'query_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'passage_input_ids': tensor([[ 101, 2178, 2177,  ...,    0,    0,    0],
        [ 101, 2023, 4552,  ...,    0,    0,    0],
        [ 101, 1037, 7185,  ...,    0,    0,    0],
        ...,
        [ 101, 4971, 1005,  ...,    0,    0,    0],
        [ 101, 2061, 3334,  ...,    0,    0,    0],
        [ 101, 4291, 4290,  ...,    0,    0,    0]]), 'passage_attention_mask': 

In [ ]:


from datasets import load_dataset

class DatasetHandler:
    """
    Class to handle loading and preprocessing of various datasets for RAG model training.
    """

    def __init__(self, dataset_name: str, version: str = None):
        """
        Initializes the DatasetHandler with the dataset name and optional version.

        Args:
            dataset_name (str): Name of the dataset (e.g., 'squad').
            version (str, optional): Specific version of the dataset (e.g., '2.0' for SQuAD 2.0).
        """
        self.dataset_name = dataset_name
        self.version = version
        self.dataset = None

    def load(self):
        """
        Loads the specified dataset using the Hugging Face datasets library.

        Raises:
            ValueError: If the dataset name is not recognized.
        """
        try:
            if self.dataset_name == 'squad' and self.version == '2.0':
                self.dataset = load_dataset('squad_v2')
            elif self.dataset_name == 'squad':
                self.dataset = load_dataset('squad')
            else:
                # Implement loading for other datasets here as needed
                raise ValueError(f"Unsupported dataset: {self.dataset_name} version: {self.version}")

            print(f"Loaded {self.dataset_name} version {self.version} dataset successfully.")

        except Exception as e:
            print(f"Error loading dataset: {e}")
            raise

    def get_train_data(self):
        """
        Returns the training data from the loaded dataset.

        Returns:
            Dataset: The training split of the dataset.
        """
        if self.dataset is None:
            raise ValueError("Dataset is not loaded. Call the load() method first.")

        return self.dataset['train']

    def get_validation_data(self):
        """
        Returns the validation data from the loaded dataset.

        Returns:
            Dataset: The validation split of the dataset.
        """
        if self.dataset is None:
            raise ValueError("Dataset is not loaded. Call the load() method first.")

        return self.dataset['validation']
